# Creating asset summaries

We'll need a *LINZ extension schema validator:*

In [1]:
from json import dumps, load
from sys import stderr
from urllib.request import urlopen

from IPython.display import JSON
from jsonschema import Draft7Validator

with urlopen("https://stac.linz.govt.nz/v0.0.10/linz/schema.json") as schema_pointer:
    schema = load(schema_pointer)

validator = Draft7Validator(schema)

def validate(instance: str) -> None:
    found_error = False
    
    for error in validator.iter_errors(instance):
        found_error = True
        print(error.message, file=stderr)
    
    if not found_error:
        print("Validated successfully")
        

The *collection example* is valid:

In [2]:
with urlopen("https://stac.linz.govt.nz/v0.0.10/linz/examples/collection.json") as example_pointer:
    example = load(example_pointer)

validate(example)

Validated successfully


Can we *summarise assets?*

In [3]:
example["summaries"]["assets"] = {
    "created": {"minimum": "1999-01-01T00:00:00Z", "maximum": "2010-01-01T00:00:00Z"},
    "updated": {"minimum": "1999-01-02T00:00:00Z", "maximum": "2010-01-02T00:00:00Z"},
}
validate(example)

Validated successfully


Yes, but *are such properties validated as normal summaries?*

In [4]:
example["summaries"]["assets"] = {
    "created": None,
    "updated": {"minimum": "1999-01-02T00:00:00Z"},
}
validate(example)

Validated successfully


Oops, that should've caused *validation exceptions!* Let's instead extend the schema to validate asset summaries as normal summaries:

In [5]:
summaries_schema = schema["definitions"]["fields"]["properties"]["summaries"]
summaries_schema["properties"]["assets"] = {
    "additionalProperties": {
        "$ref": "https://schemas.stacspec.org/v1.0.0/collection-spec/json-schema/collection.json#definitions/summaries/additionalProperties"
    }
}

validate(example)

None is not valid under any of the given schemas
{'minimum': '1999-01-02T00:00:00Z'} is not valid under any of the given schemas


Looks like that worked. For the LINZ schema we want to be even stricter. Let's start by requiring the `assets` property:

In [6]:
summaries_schema["required"] = summaries_schema.get("required", []) + ["assets"]

del example["summaries"]["assets"]
validate(example)

'assets' is a required property


Good! We also want to require `created` and `updated`:

In [7]:
summaries_schema["properties"]["assets"]["required"] = ["created", "updated"]

example["summaries"]["assets"] = {}
validate(example)

'created' is a required property
'updated' is a required property
{'stac_version': '1.0.0', 'stac_extensions': ['https://linz.github.io/stac/v0.0.10/linz/schema.json', 'https://linz.github.io/stac/v0.0.10/quality/schema.json', 'https://stac-extensions.github.io/file/v2.0.0/schema.json', 'https://stac-extensions.github.io/projection/v1.0.0/schema.json', 'https://stac-extensions.github.io/version/v1.0.0/schema.json'], 'type': 'Collection', 'id': 'collection', 'title': 'A title', 'description': 'A description', 'license': 'Apache-2.0', 'linz:lifecycle': 'under development', 'linz:providers': [{'name': 'Example', 'description': 'Example description.', 'roles': ['custodian'], 'url': 'https://www.exampleurl.com'}, {'name': 'Example', 'description': 'Example description.', 'roles': ['manager'], 'url': 'https://www.exampleurl.com'}], 'linz:security_classification': 'unclassified', 'extent': {'spatial': {'bbox': [[172.9, 1.3, 173, 1.4]]}, 'temporal': {'interval': [['2015-06-23T00:00:00Z', None]

Finally we want to make sure `minimum` and `maximum` are datetimes. At this point we've basically reimplemented `summaries/additionalProperties`, so we can delete that:

In [8]:
summaries_schema["properties"]["assets"].pop("additionalProperties", None)
summaries_schema["properties"]["assets"]["properties"] = {
    "created": {
        "type": "object",
        "required": ["minimum", "maximum"],
        "properties": {
            "minimum": {
                "type": "string",
                "format": "date-time",
                "pattern": "(\\+00:00|Z)$"
            },
            "maximum": {
                "type": "string",
                "format": "date-time",
                "pattern": "(\\+00:00|Z)$"
            }
        }
    },
    "updated": {
        "type": "object",
        "required": ["minimum", "maximum"],
        "properties": {
            "minimum": {
                "type": "string",
                "format": "date-time",
                "pattern": "(\\+00:00|Z)$"
            },
            "maximum": {
                "type": "string",
                "format": "date-time",
                "pattern": "(\\+00:00|Z)$"
            }
        }
    }
}

example["summaries"]["assets"] = {
    "created": {"minimum": "created minimum", "maximum": "created maximum"},
    "updated": {"minimum": "updated minimum", "maximum": "updated maximum"}
}
validate(example)

'created minimum' does not match '(\\+00:00|Z)$'
'created maximum' does not match '(\\+00:00|Z)$'
'updated minimum' does not match '(\\+00:00|Z)$'
'updated maximum' does not match '(\\+00:00|Z)$'


Let's make sure by reverting to the valid summary:

In [9]:
example["summaries"]["assets"] = {
    "created": {"minimum": "1999-01-01T00:00:00Z", "maximum": "2010-01-01T00:00:00Z"},
    "updated": {"minimum": "1999-01-02T00:00:00Z", "maximum": "2010-01-02T00:00:00Z"},
}
validate(example)

Validated successfully


And we're golden! The final summaries schema looks like this:

In [10]:
JSON(summaries_schema)

<IPython.core.display.JSON object>